In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

from peft import PeftModel
DEVICE = "mps" # ← adjust to "cpu", "cuda" etc. (I tested cpu, cuda, mps)

In [2]:
base = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-3B-Instruct", device_map=DEVICE)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model = PeftModel.from_pretrained(base, "models/finetuned_smaller").to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained("models/finetuned_smaller", use_fast=True)

In [4]:
test_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
You are a scientist with advanced knowledge in philosophy and social sciences.
Please, write next paragraph for the following text.

### Text:
{}

### Response:
"""

In [5]:
question = "Moral reasoning is complicated."

In [7]:
inputs = tokenizer(
    [test_prompt_style.format(question) + tokenizer.eos_token],
    return_tensors="pt"
).to(DEVICE)

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
    do_sample=True,           # ← enable sampling
    temperature=0.7,          # ← randomness control, 0.7–1.0 works well
    top_k=50,                 # ← limit sampling to top 50 tokens
    top_p=0.9,                # ← or use nucleus sampling
    repetition_penalty=1.2,   # ← discourage loops and repeats
)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(response[0].split("### Response:")[1])



## The Philosophy of Moral Reasoning

Philosophical work on moral reasoning has been ongoing since antiquity. In particular there have often been two main areas of research: one area focuses on issues related to ethics proper; another concentrates on philosophical theories or practical problems encountered by people who make decisions about ethical matters. Ethical theorists frequently emphasize three different themes (although not all philosophers necessarily agree): first, they argue that morality must be founded upon reasonableness rather than rule-following; secondly, as a matter of principle it ought to apply equally to everyone regardless of whether others follow its dictates either because they accept them or do so out of obligation; thirdly, the idea is that we need justification even when our intuitions don't support us – e.g., why does a person feel wronged after she's cheated? This article will focus primarily on the latter theme — how moral reasons should justify actions 